<h1> Coursera IBM Capstone Project </h1>

<h3> Correlation between CIty Quality of Life and Surrounding Venues </h3>

<h2> Introduction </h2>

<h4> Business Problem </h4>
A client who runs a sunccessful business, has asked for us to identify cities where their business could be equally successful.
In this project we will try to find similar cities to a selected city ('Auckland' in New Zealand) - which is where the clients business is located.

To identify possible cities, we will base this decision based on the correlation between the City Quality of Life Index and compare similar cities based on location data from FourSquare.
The quality of life index will give a glimpse into the general sentiment of our clients customers, while the FourSquare location data will provide insight into the typical city venue make-up.

<h4> Target Audience </h4>
The target audience is a client looking to identify new expansion options around the world.
They will have an idea of a current city where their business is currently successful.


<h2> Data </h2>

Based on our problem, we have identified an external data source that contains various quality of life indicies. In addition, FourSquare data will be used to identify cities with similar surrounding venues.

The following data sources will be used:
<li> City Quality of Life Dataset (from Teleport.org) </li>
<li> FourSquare location data, and venue information </li>

We will use the Quality of Life dataset identify similar rated cities (through the use of DBSCAN CLustering). Next we will use FourSquare venue data from the city centre - which will be used to identify the most similar cities from the quality of life cluster.

<h4> City Quality of Life Dataset </h4>
The quality of life is identified by several unique scores (housing, cost of living, startups, venture capital, travel connectivity, commute, business freedom, safety, healthcare, education, environmental quality, economy, taxation and internet access). By taking an average of the following scores:
<li> Cost of living </li>
<li> Startups </li>
<li> Travel Connectivity </li>
<li> Business Freedom </li>
<li> Economy </li>
<li> Taxation </li>
Using the above indicies, a clustering technique (DBSCAN or SVM) will be used to identify cities similar to the clients input option.

<h4> FourSquare Data </h4>
The FourSquare data will be used to identify the top options from the identified Quality of Life city.
This will also be selected using an un-supervised classification method.

<h4> Example Data </h4>
The Quality of Life Dataset could identify cities whose residents value similar lifestyle qualities.

Following this, we will have a list of cities that we can then compare based on similar popular venue types.